In [1]:
import numpy as np
import pandas as pd

In [ ]:
test_df = pd.read_csv('data/extract/meme_test_data_with_text.csv')
train_df = pd.read_csv('data/extract/meme_train_data_with_text.csv')

train_df.head()

,Image_name,Label,Processed_Text
0,train0001.jpg,Political,হরেক রকম চোর দিয়া বানাইছো মেলা পিসি তোমার দরবা...
1,train0002.jpg,NonPolitical,বর যাত্রী যাওয়ার টাইমে যখব গাড়িতে জায়গা কম পড়ে...
2,train0003.jpg,NonPolitical,তুমিকেখা নাবলল বুকে অনেক ব্যখাহেয় ঠিক মত পানি ...
3,train0004.jpg,NonPolitical,যখন ভল্লুক এসে পড়ে এবং আপন এবং আপনার রেস্টফরেন...
4,train0005.jpg,NonPolitical,বন্ধুযখন তার গফকেকাজিন বলে পরিচয় দেয় সম্পর্কবদ...


In [3]:
train_df.head()

,Image_name,Label,Processed_Text
0,train0001.jpg,Political,হরেক রকম চোর দিয়া বানাইছো মেলা পিসি তোমার দরবা...
1,train0002.jpg,NonPolitical,বর যাত্রী যাওয়ার টাইমে যখব গাড়িতে জায়গা কম পড়ে...
2,train0003.jpg,NonPolitical,তুমিকেখা নাবলল বুকে অনেক ব্যখাহেয় ঠিক মত পানি ...
3,train0004.jpg,NonPolitical,যখন ভল্লুক এসে পড়ে এবং আপন এবং আপনার রেস্টফরেন...
4,train0005.jpg,NonPolitical,বন্ধুযখন তার গফকেকাজিন বলে পরিচয় দেয় সম্পর্কবদ...


In [ ]:
import re
from bnunicodenormalizer import Normalizer
from spellchecker import SpellChecker

# Initialize normalizer for Bengali text
bnorm = Normalizer()

# Initialize spell checker for English
spell = SpellChecker()

def preprocess_text(text):
    """
    Preprocess extracted text (Bengali and English)
    - Remove special characters and noise
    - Normalize Bengali text
    - Fix common English misspellings
    """
    if not isinstance(text, str) or text == '':
        return ""
    
    # Convert to lowercase first
    text = text.lower()
    
    # Remove OCR noise patterns: ||, {}, [], (), ;, multiple symbols
    text = re.sub(r'[\|\{\}\[\]\(\);]+', ' ', text)
    
    # Remove patterns like '0_1', '323_4', '1r420', '4_41' (numbers with underscores or mixed)
    text = re.sub(r'\b\w*\d+_\w*\b', ' ', text)  # number_word patterns
    text = re.sub(r'\b\d+_\d+\b', ' ', text)      # digit_digit patterns
    text = re.sub(r'\b\w*\d+\w*_\b', ' ', text)   # patterns ending with underscore
    text = re.sub(r'\b_\d+\w*\b', ' ', text)      # patterns starting with underscore
    
    # Remove standalone numbers and mixed alphanumeric noise (e.g., "98", "7!!", "718")
    text = re.sub(r'\b\d+[a-z]*\b', ' ', text)
    text = re.sub(r'\b[a-z]*\d+\b', ' ', text)
    
    # Remove Bengali digits patterns like ০০, ১কির, ২জ, ২য়, ৫৫, ৬ত, য০
    # Bengali digits: ০-৯ (U+09E6 to U+09EF)
    text = re.sub(r'\b[\u09E6-\u09EF]+\b', ' ', text)  # standalone Bengali digits like ০০, ৫৫
    text = re.sub(r'\b[\u09E6-\u09EF]+[\u0980-\u09FF]+\b', ' ', text)  # digit+Bengali like ১কির, ২জ, ২য়, ৬ত
    text = re.sub(r'\b[\u0980-\u09FF]+[\u09E6-\u09EF]+\b', ' ', text)  # Bengali+digit like য০
    text = re.sub(r'\b[\u0980-\u09FF]*[\u09E6-\u09EF]+[\u0980-\u09FF]+[\u09E6-\u09EF]*\b', ' ', text)  # mixed like ০০০কৈরমনীজনিলোগতেছ
    
    # Remove excessive punctuation (keep basic ones: . , ! ?)
    text = re.sub(r'[^\w\s\u0980-\u09FF.,!?]', ' ', text)
    
    # Remove standalone single characters that are likely noise (both English and Bengali)
    text = re.sub(r'\b[a-z]\b', ' ', text, flags=re.IGNORECASE)
    # Remove single Bengali characters (like হ, ক, স, etc.)
    text = re.sub(r'\b[\u0980-\u09FF]\b', ' ', text)

    # Normalize Bengali text and fix English misspellings word by word
    words = text.split()
    cleaned_words = []
    
    for word in words:
        # Skip if too short or all punctuation
        if len(word) < 2 or word.strip('.,!?') == '':
            continue
        
        # Skip words with underscores or weird number patterns
        if '_' in word or re.search(r'\d', word):
            continue
        
        # Skip if contains Bengali digits
        if re.search(r'[\u09E6-\u09EF]', word):
            continue
        
        # Check if word contains Bengali characters
        has_bengali = bool(re.search(r'[\u0980-\u09FF]', word))
        
        if has_bengali:
            # Normalize Bengali text
            try:
                normalized = bnorm(word)['normalized']
                if normalized and len(normalized) > 1:
                    cleaned_words.append(normalized)
            except:
                # Keep original if normalization fails
                if len(word) > 1:
                    cleaned_words.append(word)
        else:
            # English word - fix common misspellings
            word_clean = word.strip('.,!?')
            if len(word_clean) > 2:  # Skip very short words
                try:
                    # Correct spelling if confidence is high
                    corrected = spell.correction(word_clean)
                    if corrected and corrected != word_clean:
                        # Only use correction if it's a known word
                        if corrected in spell:
                            cleaned_words.append(corrected)
                        else:
                            cleaned_words.append(word_clean)
                    else:
                        cleaned_words.append(word_clean)
                except:
                    cleaned_words.append(word_clean)
    
    text = ' '.join(cleaned_words)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Apply preprocessing
print("Starting text preprocessing...")
train_df['Processed_Text'] = train_df['Extracted_Text'].apply(preprocess_text)
test_df['Processed_Text'] = test_df['Extracted_Text'].apply(preprocess_text)

# Save cleaned datasets
train_df.to_csv('data/process/train_dataset_cleaned.csv', index=False)
test_df.to_csv('data/process/test_dataset_cleaned.csv', index=False)

Starting text preprocessing...


In [5]:
train_df.head()

,Image_name,Label,Processed_Text
0,train0001.jpg,Political,হরেক রকম চোর দিয়া বানাইছো মেলা পিসি তোমার দরবা...
1,train0002.jpg,NonPolitical,বর যাত্রী যাওয়ার টাইমে যখব গাড়িতে জায়গা কম পড়ে...
2,train0003.jpg,NonPolitical,তুমিকেখা নাবলল বুকে অনেক ব্যখাহেয় ঠিক মত পানি ...
3,train0004.jpg,NonPolitical,যখন ভল্লুক এসে পড়ে এবং আপন এবং আপনার রেস্টফরেন...
4,train0005.jpg,NonPolitical,বন্ধুযখন তার গফকেকাজিন বলে পরিচয় দেয় সম্পর্কবদ...
